In [62]:
import json
import pandas as pd
import ast
from typing import Dict
from datetime import datetime
import numpy as np
from fastapi import FastAPI
from starlette.testclient import TestClient
import re
from collections import Counter
import random

Previo al ETL se exploraron los archivos json para ver como estan estructurados, sin codigo.

En este ETL se extraen los datos de origen, transformandolos en base a las 5 consultas o funciones requeridas. Estas funciones se encuentran cargadas en un script aparte listo para ser deployado.

Se omiten algunos bloques de codigo referentes a la conversión de archivos CSV, JSON...


In [114]:
# Ruta al archivo JSON
archivo = 'output_steam_games.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(json.loads(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Asegúramos de que 'release_date' sea un objeto datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce', format='%Y-%m-%d')

# Calcular las horas desde el lanzamiento hasta la fecha actual
now = datetime.now()
df['hours_since_release'] = (now - df['release_date']).dt.total_seconds() / 3600

# Eliminar filas con valores NaN o inexistentes
df = df.dropna()

In [112]:
# Contar cuántos juegos se lanzaron cada año
df['release_year'] = df['release_date'].dt.year
games_per_year = df['release_year'].value_counts().sort_index()

print(games_per_year)

release_year
1983       1
1984       2
1985       1
1986       1
1987       4
1988       5
1989       7
1990       8
1991      10
1992      13
1993      24
1994      31
1995      30
1996      34
1997      38
1998      48
1999      41
2000      34
2001      55
2002      39
2003      73
2004      62
2005      85
2006     139
2007     140
2008     182
2009     300
2010     379
2011     512
2012    1081
2013    1232
2014    1973
2015    3507
2016    5105
2017    7237
2018      61
2019       1
2021       1
Name: count, dtype: int64


In [111]:
def PlayTimeGenre(genero: str) -> Dict[str, int]:
    """
    Esta función toma un género como entrada y devuelve el año de lanzamiento con más horas jugadas para ese género.
    
    Parámetros:
    genero (str): El género del juego.
    
    Devuelve:
    dict: Un diccionario con el año de lanzamiento con más horas jugadas para el género dado.
    """
    # Aseguramos de que todos los valores en 'genres' sean listas
    df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])
    
    # Filtrar el DataFrame por el género especificado
    df_genre = df[df['genres'].apply(lambda x: genero in x)]
    
    # Agrupar por año de lanzamiento y sumar las horas desde el lanzamiento
    df_genre_year = df_genre.groupby(df_genre['release_date'].dt.year)['hours_since_release'].sum()
    
    # Encontrar el año de lanzamiento con más horas desde el lanzamiento
    max_year = df_genre_year.idxmax()
    
    return {f"Año de lanzamiento con más horas jugadas para {genero}" : max_year}

resultado = PlayTimeGenre("Casual")
print(resultado)


{'Año de lanzamiento con más horas jugadas para Casual': 2017}


Construcción API

1ra Función

Reduciremos en tamaño el archivo output_steam_games.json lo más que se pueda

In [66]:
data = []

# Cargar el archivo JSON línea por línea
with open('output_steam_games.json', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Convertir a DataFrame de pandas
df = pd.DataFrame(data)

# Eliminar las filas con todas las columnas en blanco
df = df.dropna(how='all')

# Guardar el DataFrame limpio de nuevo como JSON
df.to_json('output_steam_games_limpio.json', orient='records')

In [67]:
archivo = 'output_steam_games_limpio.json'

# Lista para guardar cada fila
lista = []

with open(archivo, 'r', encoding='utf-8') as file:
    """
    Esta función lee un archivo JSON y lo convierte en una lista de diccionarios.
    """
    data = file.read()
    if data[0] == '[':
        # Los datos están en formato de array
        lista = json.loads(data)
    else:
        # Los datos están separados por nuevas líneas
        for line in data.splitlines():
            lista.append(json.loads(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Aseguramos de que 'release_date' sea un objeto datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce', format='%Y-%m-%d')

# Calcular las horas desde el lanzamiento hasta la fecha actual
now = datetime.now()
df['hours_since_release'] = (now - df['release_date']).dt.total_seconds() / 3600

app = FastAPI()

@app.get("/PlayTimeGenre/{genero}")
def PlayTimeGenre(genero: str) -> Dict[str, int]:
    """
    Esta función toma un género como entrada y devuelve el año de lanzamiento con más horas jugadas para ese género.
    
    Parámetros:
    genero (str): El género del juego.
    
    Devuelve:
    dict: Un diccionario con el año de lanzamiento con más horas jugadas para el género dado.
    """
    # Aseguramos de que todos los valores en 'genres' sean listas
    df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])
    
    # Filtrar el DataFrame por el género especificado
    df_genre = df[df['genres'].apply(lambda x: genero in x)]
    
    # Agrupar por año de lanzamiento y sumar las horas desde el lanzamiento
    df_genre_year = df_genre.groupby(df_genre['release_date'].dt.year)['hours_since_release'].sum()
    
    # Encontrar el año de lanzamiento con más horas desde el lanzamiento
    max_year = df_genre_year.idxmax()
    
    return {f"Año de lanzamiento con más horas jugadas para {genero}" : max_year}

# Crear un cliente de prueba
client = TestClient(app)

# Ahora podemos hacer solicitudes a tu API
response = client.get("/PlayTimeGenre/Simulation")
print(response.json())


{'Año de lanzamiento con más horas jugadas para Simulation': 2017}


2da Función

Tomamos australian_user_reviews.json --> 'user_id': '76561197970982479' y 'reviews' --> 'item_id': '22200' y 'posted'
Asumiremos que el usuario ha jugado desde que comento la reseña hasta la fecha de hoy

In [68]:
archivo = 'user_reviews.json/australian_user_reviews.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(ast.literal_eval(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Lista para guardar las filas del nuevo DataFrame
filas = []

# Extraer solo el campo 'recommend' de cada revisión
for index, row in df.iterrows():
    user_id = row['user_id']
    for review in row['reviews']:
        item_id = review['item_id']
        posted = review['posted']
        
        # Extraer la fecha del string 'posted' usando una expresión regular
        date_match = re.search(r'Posted ([A-Za-z]+ \d+(, \d+)?)', posted)
        if date_match:
            date_str = date_match.group(1)
            # Intentar convertir la fecha en un objeto datetime
            try:
                posted_date = datetime.strptime(date_str, '%B %d, %Y')
            except ValueError:
                try:
                    # Si no se puede convertir con el año, intentar sin el año
                    posted_date = datetime.strptime(date_str, '%B %d')
                except ValueError:
                    # Si el día está fuera de rango para el mes, continuar con la siguiente fila
                    continue
        
        # Añadir la fila a la lista de filas
        filas.append({'user_id': user_id, 'item_id': item_id, 'posted_date': posted_date})

# Crear el DataFrame a partir de la lista de filas
df_userid_item_id1 = pd.DataFrame(filas)

# Imprimir el DataFrame
print(df_userid_item_id1)


                 user_id item_id posted_date
0      76561197970982479    1250  2011-11-05
1      76561197970982479   22200  2011-07-15
2      76561197970982479   43110  2011-04-21
3                js41637  251610  2014-06-24
4                js41637  227300  2013-09-08
...                  ...     ...         ...
59275  76561198312638244      70  1900-07-10
59276  76561198312638244  362890  1900-07-08
59277        LydiaMorley  273110  1900-07-03
59278        LydiaMorley     730  1900-07-20
59279        LydiaMorley     440  1900-07-02

[59280 rows x 3 columns]


Tomamos australian_user_item.json --> 'user_id' y 'items' --> 'item_id': '10' , 'item_name': 'Counter-Strike' y 'playtime_forever' con valor en horas

In [69]:
archivo = 'australian_users_items.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    lines = file.readlines()
    # Tomar solo el 10% de las líneas
    lines = random.sample(lines, int(len(lines) * 0.1))
    for line in lines:
        lista.append(ast.literal_eval(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Lista para guardar las filas del nuevo DataFrame
filas = []

# Iterar sobre cada fila del DataFrame original
for index, row in df.iterrows():
    user_id = row['user_id']
    items = row['items']
    
    # Iterar sobre la lista de items en cada fila
    for item in items:
        item_id = item['item_id']
        item_name = item['item_name']  # Añadir item_name
        playtime_forever = item['playtime_forever']
        
        # Añadir la fila a la lista de filas
        filas.append({'user_id': user_id, 'item_id': item_id, 'item_name': item_name, 'playtime_forever': playtime_forever})

# Crear el DataFrame a partir de la lista de filas
df_userid_item_id_playtime = pd.DataFrame(filas)

# Imprimir el DataFrame resultante
print(df_userid_item_id_playtime)


                  user_id item_id                        item_name  \
0       76561198098579167   17080                   Tribes: Ascend   
1       76561198098579167  113400                     APB Reloaded   
2       76561198098579167  203290  America's Army: Proving Grounds   
3       76561198098579167  214420                          Gear Up   
4       76561198098579167  253710                        theHunter   
...                   ...     ...                              ...   
507680            Porjate  379720                             DOOM   
507681            Porjate  377160                        Fallout 4   
507682            Porjate  473770                      BallisticNG   
507683            Porjate  497850                     8-Bit Hordes   
507684            Porjate  508600           ROD: Revolt Of Defense   

        playtime_forever  
0                      0  
1                      1  
2                     11  
3                     19  
4                     15

In [70]:
df_userid_item_id_playtime.to_csv('df_userid_item_id_playtime.csv', index=False)

In [71]:
# Eliminar las filas con valores NaN en al menos una de las columnas 'user_id', 'item_id' y 'playtime_forever'
df_userid_item_id_playtime = df_userid_item_id_playtime.dropna(subset=['user_id', 'item_id', 'item_name', 'playtime_forever'], how='any')

# Imprimir el DataFrame resultante
print(df_userid_item_id_playtime)

                  user_id item_id                        item_name  \
0       76561198098579167   17080                   Tribes: Ascend   
1       76561198098579167  113400                     APB Reloaded   
2       76561198098579167  203290  America's Army: Proving Grounds   
3       76561198098579167  214420                          Gear Up   
4       76561198098579167  253710                        theHunter   
...                   ...     ...                              ...   
507680            Porjate  379720                             DOOM   
507681            Porjate  377160                        Fallout 4   
507682            Porjate  473770                      BallisticNG   
507683            Porjate  497850                     8-Bit Hordes   
507684            Porjate  508600           ROD: Revolt Of Defense   

        playtime_forever  
0                      0  
1                      1  
2                     11  
3                     19  
4                     15

In [72]:
archivo = 'output_steam_games.json'

# Lista para guardar cada fila
lista = []
with open(archivo, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(json.loads(line))

# Crear un DataFrame a partir de la lista
df = pd.DataFrame(lista)

# Lista para guardar las filas del nuevo DataFrame
filas = []

# Iterar sobre cada fila del DataFrame original
for index, row in df.iterrows():
    genres = row['genres']
    title = row['title']
        
    # Añadir la fila a la lista de filas
    filas.append({'genres': genres, 'title': title})

# Crear el DataFrame a partir de la lista de filas
df_genres_title = pd.DataFrame(filas)

# Eliminar las filas con valores NaN en ambas columnas 'genres' y 'title'
df_genres_title = df_genres_title.dropna(subset=['genres', 'title'], how='all')


# Imprimir el DataFrame resultante
print(df_genres_title)

                                                   genres  \
88310       [Action, Casual, Indie, Simulation, Strategy]   
88311                [Free to Play, Indie, RPG, Strategy]   
88312   [Casual, Free to Play, Indie, Simulation, Sports]   
88313                         [Action, Adventure, Casual]   
88315                     [Action, Adventure, Simulation]   
...                                                   ...   
120439                 [Action, Adventure, Casual, Indie]   
120440              [Casual, Indie, Simulation, Strategy]   
120441                          [Casual, Indie, Strategy]   
120442                        [Indie, Racing, Simulation]   
120443                                    [Casual, Indie]   

                           title  
88310        Lost Summoner Kitty  
88311                  Ironbound  
88312    Real Pool 3D - Poolians  
88313                    弹炸人2222  
88315      Battle Royale Trainer  
...                          ...  
120439              Ke

Union de los 2 datasets

In [73]:
# Unir los DataFrames en las columnas 'user_id' y 'item_id'
df_merged_df_userid_item_id_playtime_df_userid_item_id1 = pd.merge(df_userid_item_id_playtime, df_userid_item_id1, on=['user_id', 'item_id'], how='inner')

# Imprimir el DataFrame resultante
print(df_merged_df_userid_item_id_playtime_df_userid_item_id1)


                user_id item_id                         item_name  \
0     76561198098579167  242760                        The Forest   
1             Pdhelliot  212680            FTL: Faster Than Light   
2           Kill3rVen0m     730  Counter-Strike: Global Offensive   
3     76561198083005050    8930        Sid Meier's Civilization V   
4     76561198083005050  299360                      Block N Load   
...                 ...     ...                               ...   
4626               M_D_  463210                  State of Anarchy   
4627   shadowassassin17   34900                          Bad Rats   
4628   shadowassassin17  104900                    ORION: Prelude   
4629  76561198040611366  317780                      Eternal Fate   
4630            Porjate  221640                     Super Hexagon   

      playtime_forever posted_date  
0                 2225  1900-06-11  
1                 1740  2014-04-24  
2                46058  2015-01-03  
3                22368 

In [74]:
# Convertir la columna 'posted_date' a datetime
df_merged_df_userid_item_id_playtime_df_userid_item_id1['posted_date'] = pd.to_datetime(df_merged_df_userid_item_id_playtime_df_userid_item_id1['posted_date'])

# Crear un filtro para las fechas anteriores a 1990
filtro = df_merged_df_userid_item_id_playtime_df_userid_item_id1['posted_date'].dt.year < 1990

# Contar el número de filas con fechas anteriores a 1990
num_filas = df_merged_df_userid_item_id_playtime_df_userid_item_id1[filtro].shape[0]

print(f'Hay {num_filas} filas con fechas anteriores a 1990.')

Hay 857 filas con fechas anteriores a 1990.


In [75]:
# Crear un filtro para las fechas posteriores a 1989
filtro = df_merged_df_userid_item_id_playtime_df_userid_item_id1['posted_date'].dt.year >= 1990

# Aplicar el filtro al DataFrame
df_merged = df_merged_df_userid_item_id_playtime_df_userid_item_id1[filtro]

# Imprimir el DataFrame resultante
print(df_merged)

                user_id item_id                         item_name  \
1             Pdhelliot  212680            FTL: Faster Than Light   
2           Kill3rVen0m     730  Counter-Strike: Global Offensive   
3     76561198083005050    8930        Sid Meier's Civilization V   
4     76561198083005050  299360                      Block N Load   
5     76561198083005050  298630                     The Escapists   
...                 ...     ...                               ...   
4625               M_D_  234140                           Mad Max   
4627   shadowassassin17   34900                          Bad Rats   
4628   shadowassassin17  104900                    ORION: Prelude   
4629  76561198040611366  317780                      Eternal Fate   
4630            Porjate  221640                     Super Hexagon   

      playtime_forever posted_date  
1                 1740  2014-04-24  
2                46058  2015-01-03  
3                22368  2014-08-07  
4                 1469 

Unión con el DF que falta (genres - title)

In [76]:
# Unir los DataFrames en las columnas 'item_name' y 'title'
df_merged1 = pd.merge(df_merged, df_genres_title, left_on='item_name', right_on='title', how='inner')

# Imprimir el DataFrame resultante
print(df_merged1)

                user_id item_id                        item_name  \
0             Pdhelliot  212680           FTL: Faster Than Light   
1                 Wesmi  212680           FTL: Faster Than Light   
2           mrsmeowkins  212680           FTL: Faster Than Light   
3     76561198092114416  212680           FTL: Faster Than Light   
4              NautaIis  212680           FTL: Faster Than Light   
...                 ...     ...                              ...   
2724            jisordy  224500                         Gnomoria   
2725   consider_suicide   41700  S.T.A.L.K.E.R.: Call of Pripyat   
2726  76561198069934081   40700                      Machinarium   
2727   GoodDogRealCheap  298360                 Immortal Defense   
2728   GoodDogRealCheap  329830                  The Moon Sliver   

      playtime_forever posted_date                         genres  \
0                 1740  2014-04-24  [Indie, Simulation, Strategy]   
1                 2568  2015-06-18  [Indie, S

In [77]:
df_merged1.to_csv('2dafuncion.csv', index=False)

In [78]:
# Convertir el DataFrame a JSON
json_obj = df_merged1.to_json(orient='records')

# Imprimir el objeto JSON
print(json_obj)

# Exportar el objeto JSON a un archivo
with open('2dafuncion.json', 'w') as f:
    f.write(json_obj)

[{"user_id":"Pdhelliot","item_id":"212680","item_name":"FTL: Faster Than Light","playtime_forever":1740,"posted_date":1398297600000,"genres":["Indie","Simulation","Strategy"],"title":"FTL: Faster Than Light"},{"user_id":"Wesmi","item_id":"212680","item_name":"FTL: Faster Than Light","playtime_forever":2568,"posted_date":1434585600000,"genres":["Indie","Simulation","Strategy"],"title":"FTL: Faster Than Light"},{"user_id":"mrsmeowkins","item_id":"212680","item_name":"FTL: Faster Than Light","playtime_forever":5435,"posted_date":1436486400000,"genres":["Indie","Simulation","Strategy"],"title":"FTL: Faster Than Light"},{"user_id":"76561198092114416","item_id":"212680","item_name":"FTL: Faster Than Light","playtime_forever":6604,"posted_date":1394841600000,"genres":["Indie","Simulation","Strategy"],"title":"FTL: Faster Than Light"},{"user_id":"NautaIis","item_id":"212680","item_name":"FTL: Faster Than Light","playtime_forever":1068,"posted_date":1396483200000,"genres":["Indie","Simulation",

In [107]:
def UserForGenre(df_merged1, genero):
    """
    Esta función toma un DataFrame y un género como entrada, y devuelve el usuario con más horas jugadas para ese género,
    así como las horas jugadas por año.

    Parámetros:
    df_merged1 (pd.DataFrame): El DataFrame de entrada.
    genero (str): El género del juego.

    Devuelve:
    dict: Un diccionario con el usuario con más horas jugadas para el género dado y las horas jugadas por año.
    """
    # Filtrar el DataFrame por género
    df_genre = df_merged1[df_merged1['genres'].apply(lambda x: genero in x if isinstance(x, list) else False)]
    
    # Agrupar por 'user_id' y sumar 'playtime_forever'
    df_grouped = df_genre.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Encontrar el usuario con más horas jugadas
    max_playtime_user = df_grouped[df_grouped['playtime_forever'] == df_grouped['playtime_forever'].max()]['user_id'].values[0]
    
    # Crear un DataFrame con las horas jugadas por año
    df_genre['year'] = df_genre['posted_date'].dt.year
    df_hours_per_year = df_genre.groupby('year')['playtime_forever'].sum().reset_index()
    hours_per_year = df_hours_per_year.to_dict('records')
    
    return {"Usuario con más horas jugadas para Género X": max_playtime_user, "Horas jugadas": hours_per_year}

# Prueba la función con el género "Action"
resultado = UserForGenre(df_merged1, "Simulation")

# Imprime el resultado
print(resultado)


{'Usuario con más horas jugadas para Género X': '76561198059622971', 'Horas jugadas': [{'year': 2011, 'playtime_forever': 90095}, {'year': 2012, 'playtime_forever': 24953}, {'year': 2013, 'playtime_forever': 991458}, {'year': 2014, 'playtime_forever': 2851558}, {'year': 2015, 'playtime_forever': 3056237}]}


C:\Users\MW6529\AppData\Local\Temp\ipykernel_47152\2084833676.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre['year'] = df_genre['posted_date'].dt.year


In [106]:
ruta_archivo = '2dafuncion.csv'

# Leer el archivo CSV y convertirlo a DataFrame
df_merged2 = pd.read_csv(ruta_archivo)

print(df_merged2)

                user_id  item_id                        item_name  \
0             Pdhelliot   212680           FTL: Faster Than Light   
1                 Wesmi   212680           FTL: Faster Than Light   
2           mrsmeowkins   212680           FTL: Faster Than Light   
3     76561198092114416   212680           FTL: Faster Than Light   
4              NautaIis   212680           FTL: Faster Than Light   
...                 ...      ...                              ...   
2724            jisordy   224500                         Gnomoria   
2725   consider_suicide    41700  S.T.A.L.K.E.R.: Call of Pripyat   
2726  76561198069934081    40700                      Machinarium   
2727   GoodDogRealCheap   298360                 Immortal Defense   
2728   GoodDogRealCheap   329830                  The Moon Sliver   

      playtime_forever posted_date                               genres  \
0                 1740  2014-04-24  ['Indie', 'Simulation', 'Strategy']   
1                 256

3ra Funcion

In [82]:
# Cargamos el DataFrame desde un archivo csv
df_itemid_itemname = pd.read_csv('df_userid_item_id_playtime.csv')

# Eliminamos las columnas 'user_id' y 'playtime_forever'
df_itemid_itemname = df_itemid_itemname.drop(['user_id', 'playtime_forever'], axis=1)

In [83]:
# Supongamos que quieres conservar el 0.5% de los datos
porcentaje = 0.005
df_itemid_itemnamem0005 = df_itemid_itemname.sample(frac=porcentaje)

In [84]:
print(df_itemid_itemnamem0005)

        item_id                               item_name
396444      550                           Left 4 Dead 2
285763    65930           The Bureau: XCOM Declassified
482892   377070                             Noctropolis
15896    234310                            March of War
320764    47810  Dragon Age: Origins - Ultimate Edition
...         ...                                     ...
424396   219740                            Don't Starve
269367    47780                            Dead Space 2
36694    283370  Marine Sharpshooter II: Jungle Warfare
336000    49520                           Borderlands 2
57184      4920                     Natural Selection 2

[2538 rows x 2 columns]


In [85]:
print(df_itemid_itemname)

        item_id                        item_name
0         17080                   Tribes: Ascend
1        113400                     APB Reloaded
2        203290  America's Army: Proving Grounds
3        214420                          Gear Up
4        253710                        theHunter
...         ...                              ...
507680   379720                             DOOM
507681   377160                        Fallout 4
507682   473770                      BallisticNG
507683   497850                     8-Bit Hordes
507684   508600           ROD: Revolt Of Defense

[507685 rows x 2 columns]


In [86]:
ruta_archivo = 'australian_user_reviews.json'

# Lista para almacenar los objetos
lista_objetos = []

# Abrir el archivo JSON
with open(ruta_archivo, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # Esto verifica que la línea no esté vacía
            user = ast.literal_eval(line)  # Convertir la línea en un objeto Python
            lista_objetos.append(user)  # Agregar el objeto a la lista

# Convertir la lista de objetos a un DataFrame
dfaustralian_user_reviews = pd.DataFrame(lista_objetos)

# Eliminar las columnas 'user_id' y 'user_url'
dfaustralian_user_reviews = dfaustralian_user_reviews.drop(['user_id', 'user_url'], axis=1)

In [87]:
print(dfaustralian_user_reviews)

                                                 reviews
0      [{'funny': '', 'posted': 'Posted November 5, 2...
1      [{'funny': '', 'posted': 'Posted June 24, 2014...
2      [{'funny': '', 'posted': 'Posted February 3.',...
3      [{'funny': '', 'posted': 'Posted October 14, 2...
4      [{'funny': '3 people found this review funny',...
...                                                  ...
25794  [{'funny': '', 'posted': 'Posted May 31.', 'la...
25795  [{'funny': '', 'posted': 'Posted June 17.', 'l...
25796  [{'funny': '1 person found this review funny',...
25797  [{'funny': '', 'posted': 'Posted July 21.', 'l...
25798  [{'funny': '1 person found this review funny',...

[25799 rows x 1 columns]


In [88]:
# Extraer 'item_id', 'posted' y 'recommend' de la columna 'reviews'
dfaustralian_user_reviews['item_id'] = dfaustralian_user_reviews['reviews'].apply(lambda x: x[0]['item_id'] if x else None)
dfaustralian_user_reviews['posted'] = dfaustralian_user_reviews['reviews'].apply(lambda x: x[0]['posted'] if x else None)
dfaustralian_user_reviews['recommend'] = dfaustralian_user_reviews['reviews'].apply(lambda x: x[0]['recommend'] if x else None)

# Reemplazar los valores None en 'item_id' con un valor por defecto
dfaustralian_user_reviews['item_id'].fillna('0', inplace=True)

In [89]:
# Convertir 'item_id' a int64 en ambos DataFrames
dfaustralian_user_reviews['item_id'] = dfaustralian_user_reviews['item_id'].astype('int64')
df_itemid_itemnamem0005['item_id'] = df_itemid_itemnamem0005['item_id'].astype('int64')

In [90]:
# Unir los DataFrames
df_funcion3 = dfaustralian_user_reviews.merge(df_itemid_itemnamem0005, on='item_id', how='left')

In [91]:
# Ver el DataFrame final
print(df_funcion3)

                                                  reviews  item_id  \
0       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
1       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
2       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
3       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
4       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
...                                                   ...      ...   
168112  [{'funny': '1 person found this review funny',...   273110   
168113  [{'funny': '1 person found this review funny',...   273110   
168114  [{'funny': '1 person found this review funny',...   273110   
168115  [{'funny': '1 person found this review funny',...   273110   
168116  [{'funny': '1 person found this review funny',...   273110   

                          posted recommend                      item_name  
0       Posted November 5, 2011.      True                  Killing Floor  
1      

In [92]:
# Supongamos que 'df' es tu DataFrame
print(df_funcion3.columns)

Index(['reviews', 'item_id', 'posted', 'recommend', 'item_name'], dtype='object')


In [93]:
# Ver el DataFrame final
print(df_funcion3)

                                                  reviews  item_id  \
0       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
1       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
2       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
3       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
4       [{'funny': '', 'posted': 'Posted November 5, 2...     1250   
...                                                   ...      ...   
168112  [{'funny': '1 person found this review funny',...   273110   
168113  [{'funny': '1 person found this review funny',...   273110   
168114  [{'funny': '1 person found this review funny',...   273110   
168115  [{'funny': '1 person found this review funny',...   273110   
168116  [{'funny': '1 person found this review funny',...   273110   

                          posted recommend                      item_name  
0       Posted November 5, 2011.      True                  Killing Floor  
1      

In [94]:
df_funcion3.drop('reviews', axis=1, inplace=True)

In [95]:
print(df_funcion3.columns)

Index(['item_id', 'posted', 'recommend', 'item_name'], dtype='object')


In [96]:
print(df_funcion3)

        item_id                    posted recommend  \
0          1250  Posted November 5, 2011.      True   
1          1250  Posted November 5, 2011.      True   
2          1250  Posted November 5, 2011.      True   
3          1250  Posted November 5, 2011.      True   
4          1250  Posted November 5, 2011.      True   
...         ...                       ...       ...   
168112   273110            Posted July 3.      True   
168113   273110            Posted July 3.      True   
168114   273110            Posted July 3.      True   
168115   273110            Posted July 3.      True   
168116   273110            Posted July 3.      True   

                            item_name  
0                       Killing Floor  
1                       Killing Floor  
2                       Killing Floor  
3                       Killing Floor  
4                       Killing Floor  
...                               ...  
168112  Counter-Strike Nexon: Zombies  
168113  Counter-Str

In [109]:
def UsersRecommend(año: int):
    """
    Esta función toma un año como entrada y devuelve los tres juegos más recomendados para ese año.

    Parámetros:
    año (int): El año de interés.

    Devuelve:
    list: Una lista de diccionarios con los tres juegos más recomendados para el año dado.
    """
    # Crear un contador para los juegos
    game_counter = Counter()

    # Convertir la columna 'posted' a datetime
    df_funcion3['posted_date'] = pd.to_datetime(df_funcion3['posted'].str[7:-1], errors='coerce')

    # Filtrar las filas donde 'posted' es del año dado y 'recommend' es True
    df_filtered = df_funcion3[(df_funcion3['posted_date'].dt.year == año) & df_funcion3['recommend']]

    # Contar las ocurrencias de cada juego
    game_counter = Counter(df_filtered['item_name'])

    # Obtener los 3 juegos más comunes
    most_common_games = game_counter.most_common(3)

    # Devolver los resultados
    return [{"Puesto " + str(i+1) : most_common_games[i]} if len(most_common_games) > i else {"Puesto " + str(i+1) : None} for i in range(3)]

# Probar la función
resultado = UsersRecommend(2011)
print(resultado)


[{'Puesto 1': ("Garry's Mod", 115)}, {'Puesto 2': ('Left 4 Dead 2', 75)}, {'Puesto 3': ('Terraria', 75)}]


In [98]:
# Encontrar la fecha más reciente en la columna 'posted_date'
fecha_mas_antigua = df_funcion3['posted_date'].min()
fecha_mas_reciente = df_funcion3['posted_date'].max()

print(f"La fecha más antigua  en el dataframe es: {fecha_mas_antigua}")
print(f"La fecha más reciente en el dataframe es: {fecha_mas_reciente}")

La fecha más antigua  en el dataframe es: 2010-11-20 00:00:00
La fecha más reciente en el dataframe es: 2015-12-31 00:00:00


In [99]:
# Supongamos que quieres conservar el 10% de los datos
porcentaje = 0.1
df_funcion3m01 = df_funcion3.sample(frac=porcentaje)

In [100]:
print(df_funcion3m01)

        item_id                     posted recommend  \
72920    202970              Posted May 6.      True   
36012       730       Posted May 19, 2015.      True   
80490    304930      Posted July 31, 2014.      True   
29349      8930       Posted July 1, 2014.      True   
130900     4000   Posted February 2, 2014.      True   
...         ...                        ...       ...   
162736   304930     Posted August 4, 2014.      True   
22898       620  Posted September 6, 2014.      True   
48427    236390       Posted May 21, 2014.      True   
56684    208090    Posted October 9, 2014.      True   
69186       620  Posted September 7, 2014.      True   

                               item_name posted_date  
72920         Call of Duty: Black Ops II         NaT  
36012   Counter-Strike: Global Offensive  2015-05-19  
80490                           Unturned  2014-07-31  
29349         Sid Meier's Civilization V  2014-07-01  
130900                       Garry's Mod  2014-02-02